<a href="https://colab.research.google.com/github/rkruser/ai4all-umd-2020/blob/master/Image_Recognition_Final_Project_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Setting up the files

In this colab, you will put together everything you have learned. First, let's install necessary software and download the github repo.

In [ ]:
!pip install flask-ngrok
!git clone https://github.com/rkruser/ai4all-umd-2020.git #https seems to work, but not ssh
%cd ai4all-umd-2020

Then download the leafsnap dataset and unpack it:

In [ ]:
!mkdir data
!mkdir data/leafsnap
!curl -o ./data/leafsnap/leafsnap.tar http://leafsnap.com/static/dataset/leafsnap-dataset.tar
!tar -xvf ./data/leafsnap/leafsnap.tar -C ./data/leafsnap > /dev/null

**Finally, follow these drive links** to download 
[resnet trained on Leafsnap](https://drive.google.com/file/d/156hbvRB-EkQTkyMIjru6lye-5TUAosjF/view?usp=sharing) and [your neural network](https://drive.google.com/file/d/1YH70L-pESc8m4N-vgPjVk6YbnUIEUo6k/view?usp=sharing) trained on leafsnap. Save these files on your computer, then manually upload them to `ai4all-umd-2020/models/resnet` and `ai4all-umd-2020/models/your_model` respectively by using the folder interface on the left side of the Colab. (Expand the desired folder, then click on the three dots to the right of its name, and then click "upload").

**Uploading may take a minute. The bottom left corner of Colab should show upload progress.** You may get code errors if you try to run cells before the upload is complete.

**Note that you will have to re-upload these if Colab decides to obliterate your file system.** Manual uploads are the simplest solution for right now. The colab will not be wiped clean too often if you are active on it.

## Configuring Git
**Note** that if you're just running the colab without making updates to the github project, you can skip the configuring / using git steps.


Here we configure git so that you can push and pull to the repository from your colab. Replace {Your github email}, {Your github username}, and {your github password} with the appropriate information (delete the brackets too), then run the following code cell. When you're done, hide this cell, as it contains your github password.

In [ ]:
!git config --global user.email "{your git email}"
!git config --global user.name "{your git username}"
!git remote set-url origin "https://{your git username}:{your git password}@github.com/rkruser/ai4all-umd-2020.git"

## Using Git

"git status -s" checks whether there are changes that need to be committed

Run "git add" and "git commit" to save the changes to git when needed

Run "git push origin master" to push changes to everyone.

Run "git pull origin master" to pull changes made by other people. If there is a merge conflict, you may need to resolve it, then run a "git add" / "git commit" to save the changes.

In [ ]:
!git status
!git add app_functions.py
!git commit -m 'Added app_functions'

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	app_functions.py
	models/resnet/resnet_model_49.pth
	models/your_model/your_model_49.pth

nothing added to commit but untracked files present (use "git add" to track)
[master 3b22024] Added app_functions
 1 file changed, 175 insertions(+)
 create mode 100644 app_functions.py


In [ ]:
!git push origin master

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.21 KiB | 2.21 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/rkruser/ai4all-umd-2020.git
   49b37be..3b22024  master -> master


In [ ]:
!git pull origin master

remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
From https://github.com/rkruser/ai4all-umd-2020
 * branch            master     -> FETCH_HEAD
   3b22024..ab26f8e  master     -> origin/master
Updating 3b22024..ab26f8e
Fast-forward
 app_functions.py | 10 +++++---
 emily.py         | 73 ++++++++++++++++++++++++++------------------------------
 portia.py        | 51 +++++++++++++++++++++++++++++++++++++++
 3 files changed, 92 insertions(+), 42 deletions(-)
 create mode 100644 portia.py


# Exploring your trained networks

Let's load both the network that you designed and the standard resnet18 model. Both "your_model_49" and "resnet_model_49" have been trained on a small version of the Leafsnap dataset for 50 epochs.

Import libraries:

In [ ]:
import torch
import project_network as pnet
import project_train as ptrain
import data_loader
import numpy as np
from importlib import reload

Reload libraries only if necessary (if you've made changes after previously loading). Otherwise you can skip the next code cell.

In [ ]:
pnet = reload(pnet)
ptrain = reload(ptrain)
data_loader = reload(data_loader)

Load the networks:

In [ ]:
device = 'cpu' #We don't need to bother with the GPU when testing the trained model on small sets of images

your_net = pnet.YourNetwork(return_intermediates=True) # A special argument I added to your network --Ryen
your_net.eval() # Put in testing mode
net_weights, _ = torch.load('./models/your_model/your_model_49.pth',map_location=device) # The second return value is the optimizer weights, which we don't need now
your_net.load_state_dict(net_weights)

resnet18 = pnet.models.resnet18(pretrained=False, num_classes=185)
resnet18.eval() # Put in testing mode
resnet_weights, _ = torch.load('./models/resnet/resnet_model_49.pth',map_location=device) # The second return value is the optimizer weights, which we don't need now
resnet18.load_state_dict(resnet_weights)

<All keys matched successfully>

Now let's visualize a subset of leafsnap.

In [ ]:
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
reduce_size = transforms.Resize((600,600))
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()
leaf_species_name_mapping = data_loader.ClassLoader()

# A function to conveniently print a list as a grid
def print_list_grid(lst, nrow=8):
  to_print = '[\n['
  for count, l in enumerate(lst):
    to_print += str(l)+', '
    if (count+1)%nrow == 0:
      if count+1 < len(lst):
        to_print += ']\n['
      else:
        to_print += ']\n'
  to_print += ']'
  print(to_print)

def rescale(im_tensor, perc1 = 20, perc2 = 98):
  numpy_tensor = im_tensor.numpy()
  pc1 = np.percentile(numpy_tensor, perc1)
  pc2 = np.percentile(numpy_tensor, perc2)
  return ((im_tensor-pc1)/(pc2-pc1)).clamp(0,1)

In [ ]:
LeafSnapLoader = ptrain.LeafSnapLoader # Get objects like this so they can be easily reloaded if we change it
default_im_transform = ptrain.default_im_transform


leafsnap_test_dataset = LeafSnapLoader(mode='test',transform=default_im_transform)
image_sample = []
image_species_index = []
image_species = []

increment = len(leafsnap_test_dataset)//64

# Get 64 leafsnap data samples spaced throughout the dataset
for i in range(0, 64*increment, increment):
  sample = leafsnap_test_dataset[i]
  image_sample.append(sample['image'])
  image_species_index.append(sample['species_index'])
  image_species.append(sample['species'])

im_tensor = torch.stack(image_sample)
species_label_tensor = torch.LongTensor(image_species_index)
im_grid = make_grid(im_tensor, nrow=8)
im_grid = reduce_size(to_pil(im_grid))


In [ ]:
display(im_grid)
print_list_grid(image_species)

Now let's run the neural networks on this sample of images and see how well they did.

In [ ]:
with torch.no_grad():
  your_result, inter1, inter2 = your_net(im_tensor)
  _, your_result = torch.max(your_result, 1)

  resnet_result = resnet18(im_tensor)
  _, resnet_result = torch.max(resnet_result, 1)

In [ ]:
your_num_correct = (your_result == species_label_tensor).sum()
your_species_predictions = [leaf_species_name_mapping.ind2str(ind.item()) for ind in your_result]
your_correctness_grid = (your_result == species_label_tensor).view(8,8)

resnet_num_correct = (resnet_result == species_label_tensor).sum()
resnet_species_predictions = [leaf_species_name_mapping.ind2str(ind.item()) for ind in resnet_result]
resnet_correctness_grid = (resnet_result == species_label_tensor).view(8,8)

print("Your model got {0} out of {1}".format(your_num_correct,64))
print_list_grid(your_species_predictions)
print(your_correctness_grid)

print("\n")

print("resnet18 got {0} out of {1}".format(resnet_num_correct,64))
print_list_grid(resnet_species_predictions)
print(resnet_correctness_grid)

### Exercise

Find images in the above grid that:

* Both models classified correctly
* Your model got right but resnet got wrong
* Resnet got right but your model got wrong
* Both your models got wrong

For the wrong images, find an image in LeafSnap of the species that the model(s) mistakenly thought were the true classification. (You can find images of various species in data/leafsnap/dataset/images/field/{species_name}/). Compare these wrong images with images of the true species.

Include these examples in your presentation and see if you can explain why the network may have gotten the classification wrong.

## Visualizing convolutional activations

Let's visualize the convolutional activations of your network on one or two of the leafsnap test images.

Above, when we run your network, we extract "inter1" and "inter2". If you look inside project_network.py, you will see that these correspond to the outputs of your first convolution-relu-pooling and your second convolution-relu-pooling layers respectively.

In [ ]:
print(inter1.size())
print(inter2.size())

As you can see from the sizes printed above, each intermediate layer is a batch of 64 convolutional image activations. Each "image" in inter1 has 64 convolutional channels, and each "image" in inter2 has 128 convolutional channels. Each channel corresponds to a different convolution operation learned by the neural network.

Let's visualize the convolutions of the third image in this batch.

In [ ]:
standard_size = transforms.Resize((600,600),interpolation=0)

In [ ]:
acer_palmatum_inter1 = inter1[2].unsqueeze(1)
acer_palmatum_inter2 = inter2[2,:64].unsqueeze(1) #Take first 64 conv channels

# Unsqueeze dimension 1 to get 64 x 1 x 55 x 55 and 64 x 1 x 13 x 13
# This allows us to pretend that the convolution channels are like a batch of 64 black and white images

acer_palmatum_inter1_conv_grid = make_grid(acer_palmatum_inter1, nrow=8, normalize=True,
                                    scale_each = True)
acer_palmatum_inter2_conv_grid = make_grid(acer_palmatum_inter2, nrow=8, normalize=True,
                                    scale_each = True)

# Use the rescaling function to make the outputs look prettier
acer_palmatum_inter1_conv_grid = rescale(acer_palmatum_inter1_conv_grid)
acer_palmatum_inter2_conv_grid = rescale(acer_palmatum_inter2_conv_grid)

print("Acer_palmatum first set of conv/pooling layers")
display(standard_size(to_pil(acer_palmatum_inter1_conv_grid)))

print('\n')

print("Acer_palmatum second set of conv/pooling layers")
display(standard_size(to_pil(acer_palmatum_inter2_conv_grid)))


Note the increasing level of shape abstraction in the convolution outputs.

### Exercise

Pick a different leaf image and visualize its intermediate convolution layers in the same way.

## Accuracy on test set

Now let's run the networks on the entire test dataset and compute the accuracy.

In [ ]:
print("Testing your network")
your_net.return_intermediates = False
ptrain.test(your_net)

print('\n')

print("Testing resnet18")
ptrain.test(resnet18)

## Training results

The training results are saved in models/resnet/stats_49.pkl and models/your_model/stats_49.pkl. There are also the "print_output.txt" files in the same directory, which show all output printed to the screen during training (you can open this file in Colab to see what training looked like).

The stats_49.pkl files are python *pickle* files. That is, they are python objects saved using the pickle library. Let's load these objects for use.

In [ ]:
import pickle

your_model_train_losses, your_model_val_accuracies = pickle.load(open('./models/your_model/stats_49.pkl','rb'))
resnet_model_train_losses, resnet_model_val_accuracies = pickle.load(open('./models/resnet/stats_49.pkl','rb'))

print(your_model_train_losses)
print(your_model_val_accuracies)
print(resnet_model_train_losses)
print(resnet_model_val_accuracies)

Train_losses and val_accuracies are both lists of 50 entries, one for each training epoch. Train_losses holds the average loss for each epoch. Val_accuracies holds the percentage accuracy of the model on the validation set after each epoch. 

**Note** that val_accuracies is a list of tuples (epoch_number, accuracy), so you'll have to write a small bit of code to extract the accuracies into their own list.

### Exercise

Plot the training losses and validation accuracies versus num_epochs for each network. Make one plot for losses and one plot for accuracies, and on each plot have two lines, one for your model and one for resnet.

Example of how to plot two lines on a graph:

```
plt.xlabel("X-axis name")
plt.ylabel("Y-axis name")
plt.title("Title of figure")
plt.plot( values, label="Line label" )
plt.plot( values2, label="Line 2 label" )
plt.legend()
plt.show()
```



In [ ]:
import matplotlib.pyplot as plt

# Your plotting code here

### Exercise: Analyzing the training curves

1. After how many epochs does performance mostly stop improving for each model?
2. Which model learns faster?
3. Which model achieves higher validation accuracy?
4. Can you see any evidence of overfitting in the training curves?
5. Compare the validation accuracies with the test set accuracies from earlier in this colab. According to the test data, which model is better?
6. Why use a separate validation and test set?
7. There are 185 tree species in this dataset. If some model guesses the true classification at random, what would you expect the model accuracy to be on the test data? How well does your model do compared to a completely random model?

Note that resnet is almost certainly going to do better than your model. This is because the design of resnet is highly optimized by professional machine learning researchers, so don't feel too bad about it.

# Putting it all together in the Web app

Here we have our web app code. The app will take a user-uploaded image and:

* Run the pretrained imagenet resnet50 to predict what the object is
* Run your leafsnap model on the image in case the image is a leaf
* Run each of your individual image transforms on the input and display the results on the web page

In [ ]:
# Run resnet50 pretrained on imagenet, your neural network model,
#  and all student image transformations
#  Collect results in a dictionary
import traceback
from flask import Flask, jsonify, request, render_template
from flask_ngrok import run_with_ngrok

#from utils import read_file, transform_image, get_topk, model  #render_prediction

app = Flask(__name__)

import os
import json
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np

import project_network as pnet
import project_train as ptrain
import data_loader

from PIL import Image
import requests
from io import BytesIO
import base64

from data_loader import ClassLoader

# Student image transforms
# from Anu import ...
# from shubham import ...
# from emily import ...
# from portia import ...
# from kemka import ...
# from unity import ...


def read_file(upload=None, url=None):
    if (upload is not None) and upload.filename:
        in_memory_file = BytesIO()
        upload.save(in_memory_file)
        img = Image.open(in_memory_file)
        return img

    elif url is not None:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img

    else:
        raise NameError('Invalid file/url')

def to_base64(img):
    buffered = BytesIO()
    img.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('ascii')

# Transform input into the form our model expects
def transform_image(pil_image):
    input_transforms = [
        transforms.Resize(255),           # We use multiple TorchVision transforms to ready the image
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],       # Standard normalization for ImageNet model input
            [0.229, 0.224, 0.225]
        )
    ]
    my_transforms = transforms.Compose(input_transforms)
    timg = my_transforms(pil_image)                       # Transform PIL image to appropriately-shaped PyTorch tensor
    timg.unsqueeze_(0)                                    # PyTorch models expect batched input; create a batch of 1
    return timg

leafsnap_transform_image = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor()
])

def get_topk(model, input_tensor, k=5):
    outputs = model(input_tensor)                 # Get likelihoods for all ImageNet classes
    values, indices = torch.topk(outputs, k)              # Extract top k most likely classes
    values = values.data.cpu().numpy()[0]
    indices = indices.data.cpu().numpy()[0]
    return values, indices

resnet50_imagenet_model = models.resnet50(pretrained=True)
resnet50_imagenet_model.eval()
img_class_map = None
mapping_file_path = 'index_to_name.json'                  # Human-readable names for Imagenet classes
if os.path.isfile(mapping_file_path):
    with open (mapping_file_path) as f:
        img_class_map = json.load(f)

device = 'cpu' #We don't need to bother with the GPU when testing the trained model on small sets of images
your_net = pnet.YourNetwork() # A special argument I added to your network --Ryen
net_weights, _ = torch.load('./models/your_model/your_model_49.pth',map_location=device) # The second return value is the optimizer weights, which we don't need now
your_net.load_state_dict(net_weights)
your_net.eval()
leaf_species_name_mapping = ClassLoader()
downsize = transforms.Resize(60)

# Need to return a list of dictionaries with keys 'model', 'label', 'score', 'image'
# Ryen will write this function and get back to you
def collect_outputs(input_pil_image):
  resnet50_im = transform_image(input_pil_image)
  your_net_im = leafsnap_transform_image(input_pil_image).unsqueeze(0)

  r50_vals, r50_inds = get_topk(resnet50_imagenet_model, resnet50_im, 5)
  your_vals, your_inds = get_topk(your_net, your_net_im, 5)

  image_net_results = []
  for value, idx in zip(r50_vals, r50_inds):
    image_net_results.append({
        "model": "ImageNet Resnet50 Pretrained",
        "category": img_class_map.get(str(idx), "Unknown")[1],
        "score": str(value),
        "image": None
    })

  your_net_results = []
  for value, idx in zip(your_vals, your_inds):
    species_name = leaf_species_name_mapping.ind2str(idx)
    species_dir = os.path.join('./data/leafsnap/dataset/images/field/',species_name)
    species_file = os.listdir(species_dir)[0]
    species_file = os.path.join(species_dir, species_file)
    your_net_results.append({
        "model": "Your Network Trained on Leafsnap",
        "category": species_name,
        "score": str(value),
        "image": to_base64(downsize(Image.open(species_file)))
    })

  # In place of "None", write
  #  to_base64( student_transform( img ) )
  #  student_transform must take a PIL image and return a PIL image
  #  The returned image must be no larger than 256 by 256, preferably smaller
  student_image_transforms = [
    {
      "model": "Shubham",
      "category": '-',
      "score": '-',
      "image": None 
    },
    {
      "model": "Portia",
      "category": '-',
      "score": '-',
      "image": None # Fill this in    
    },
    {
      "model": "Kemka",
      "category": '-',
      "score": '-',
      "image": None # Fill this in    
    },
    {
      "model": "Emily",
      "category": '-',
      "score": '-',
      "image": None # Fill this in    
    },
    {
      "model": "Unity",
      "category": '-',
      "score": '-',
      "image": None # Fill this in    
    },
    {
      "model": "Anu",
      "category": '-',
      "score": '-',
      "image": None # Fill this in    
    },
  ]

  all_results = image_net_results + your_net_results + student_image_transforms

  return all_results


In [ ]:
@app.route('/', methods=['GET'])
def root():
    return render_template('index.html')

# new test comment

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'GET':
        try:
            url = request.args.get('q')
            app.logger.debug('url provided - %s', url)
            input_tensor = transform_image(read_file(url=url))
            values, indices = get_topk(input_tensor)
            results = render_prediction(values, indices)
            return jsonify(results=results)

        except:
            app.logger.debug("Error: %s", traceback.print_exc())
            return jsonify("invalid image url")

    elif request.method == 'POST':
        try:
            file = request.files['file']
            app.logger.debug('file uploaded - %s', file)
            url = request.form.get("url", None)
            app.logger.debug('url provided - %s', url)

            input_pil_image = read_file(upload=file, url=url)
            #values, indices = get_topk(input_tensor)
            results = collect_outputs(input_pil_image)
            return jsonify(results=results)

        except:
            app.logger.debug("Error: %s", traceback.print_exc())
            return jsonify("invalid image")

    else:
        app.logger.debug("Error: %s", traceback.print_exc())
        return jsonify('invalid request')

When ready to run the app, execute the following code and use the printed ngrok.io link to go to the web page.

In [ ]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://efcf2be3e571.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Jul/2020 19:02:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2020 19:02:00] "GET /static/main.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2020 19:02:00] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2020 19:02:01] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Jul/2020 19:02:45] "POST /predict HTTP/1.1" 200 -
